# Combined plots - vector and scalar map

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
%matplotlib widget

In [ ]:
import psyplot.project as psy

import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
from IPython.display import display, Video

from ipympl.backend_nbagg import Canvas
Canvas.header_visible.default_value = False

import warnings
warnings.filterwarnings("ignore", r"\s*The on_mappable_changed")
warnings.filterwarnings("ignore", r"\s*The input coordinates")
warnings.filterwarnings("ignore", r"\s*shading=")
warnings.filterwarnings("ignore", r"\s*\[Warning by")
warnings.filterwarnings("ignore", r"\s*ShapelyDeprecationWarning")

from iconarray.plot import formatoptions # import plotting formatoptions (for use with psyplot)
import iconarray as iconvis # import icon-vis self-written modules

In [ ]:
iconvis.get_example_data()

In [ ]:
f_icon      = '../data/example_data/grib/lfff00000000_uv'
gridfile = '../data/example_data/grids/opr_r19b08'

In [ ]:
# The icon file is in GRIB format, so we require the cfgrib engine to open. 
# If you look a the resulting xarray dataset you can see there is no grid data, so we need to merge it, see next cell.

ds_grib = psy.open_dataset(f_icon, engine='cfgrib', backend_kwargs={'indexpath': '', 'errors': 'ignore'})
ds_grib

In [ ]:
ds = iconvis.combine_grid_information(ds_grib, gridfile)
ds

In [ ]:
# Dont plot with x and y gridlines
psy.rcParams["plotter.maps.xgrid"] = False
psy.rcParams["plotter.maps.ygrid"] = False
psy.rcParams["plotter.maps.grid_settings"] = {'visible': False}
psy.rcParams["plotter.maps.grid_labelsize"] = 'x-small'

In [ ]:
from psy_simple.plotters import CMap, Bounds
from psy_maps.plotters import Transform, MapPlot2D, LonLatBox, MapPlotter
from psyplot.plotter import Plotter
import psy_simple.plotters
import psy_maps.plotters
import cartopy.crs as ccrs
import matplotlib.pyplot as plt

# Quiver plot

### Example with ICON data, before interpolation to regular grid

Result is mainly all black due to high density of arrows.

In [ ]:
combinedplot_1 = psy.plot.mapcombined(ds,
            name=[['t',['u','v']]], 
            generalVerticalLayer=50, time=0, datagrid={'c': 'k', 'lw': 0.1},
            borders=True, 
            cmap='Reds', 
            arrowsize=100, 
            vcbar='r',
            clabel='{desc}', 
            vclabel='Wind Speed [%(units)s]',
            #map_extent=[5.5, 11.0, 45.5, 48.0]
            )

In [ ]:
# Zoom in to small area to see that data is plotted correctly, but the plot was not useful when viewing entire region.
combinedplot_1.update(map_extent=[7.8, 8.0, 47.0, 47.1])

### Remap ICON wind data to Regular grid for plotting

ICON data can be too dense to plot wind barbs nicely, so to reduce the density of barbs, we can first interpolate the data to regular grid.

#### Fieldextra interpolation

In [ ]:
output_file = iconvis.remap_ICON_to_regulargrid(
                                        data_file=f_icon,
                                        in_grid_file=gridfile,
                                        num_dates=ds.time.shape[0], # number of time steps in your data.
                                        region='ch' # 'switzerland'/'ch' or 'europe'
                                    )

In [ ]:
print(output_file)

ds_interp = psy.open_dataset(output_file)
ds_interp

In [ ]:
ds_interp.U.encoding['coordinates'] = 'x_1 y_1'
ds_interp.V.encoding['coordinates'] = 'x_1 y_1'

#### Note
The remap_ICON_to_regulargrid module has only been tested/written for netcdf, with U,V variables, and creating a regular grid region over Switzerland). If the namelist generation doesnt work for your data, you can alternatively modify the NAMELIST_ICON_REG_REMAP file and run fieldextra yourself. For example on Tsa, in your shell you can run:

`/project/s83c/fieldextra/tsa/bin/fieldextra_gnu_opt_omp NAMELIST_ICON_REG_REMAP` 

You can define the output location and use that as input to psyplot, as in next step.

In [ ]:
import psyplot.data as psyd

uv = ds_interp.psy.create_list(name=[["U", "V"]], time=0, z=79, squeeze=True)[0] # regular grid wind data
temp = ds.psy.create_list(name="t", t=0, z=79)[0]  # icon grid temperature data


In [ ]:
data = psyd.InteractiveList([temp, uv])

In [ ]:
plotter = psy.plot.mapcombined.plotter_cls(data)

In [ ]:
from psyplot.project import Project
project = Project([data])

In [ ]:
project.update(
    borders=True,
    cmap='Reds',
    density=0.4, 
    arrowsize=200,
    map_extent=[6.9, 10.2, 45.3, 47.5],
   vplot='quiver',
clabel='{desc}',
title='Combined Scalar and Vector Plot after Interpolation')

In [ ]:
plt.ioff()

with project as sp:
    sp.export('combined_plot.png')

plt.ion()